In [ ]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# 1. Load data
df = pd.read_csv(r"../Datasets/jobs_cleaned.csv")
docs = df['e5_input'].tolist()
classes = df['Chuyên môn'].tolist() # DA, DS, MLE

# 2. Embedding với E5 (Dùng bản multilingual cho Anh-Việt)
embedding_model = SentenceTransformer('intfloat/multilingual-e5-large')
embeddings = embedding_model.encode(docs, show_progress_bar=True)

# 3. Cấu hình các thành phần (Fix lỗi ensure_all_finite bằng cách khởi tạo mới)
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="english", min_df=2)

Batches:  30%|███       | 32/105 [15:54<36:59, 30.41s/it]

In [ ]:
# Lưu models để sử dụng lại sau này
embedding_model.save(r"../Models/embedding_model")
umap_model.save(r"../Models/umap_model")
hdbscan_model.save(r"../Models/hdbscan_model")
vectorizer_model.save(r"../Models/vectorizer_model")

In [ ]:
# 4. Khởi tạo và chạy BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    nr_topics="auto"
)

topics, probs = topic_model.fit_transform(docs, embeddings)

# 5. Tính toán phân bổ theo chuyên môn (Để dùng cho đoạn code dưới)
topics_per_class = topic_model.topics_per_class(docs, classes=classes)

# 6. Trực quan hóa
fig_topics = topic_model.visualize_topics()
fig_hierarchy = topic_model.visualize_hierarchy()
fig_barchart = topic_model.visualize_barchart(top_n_topics=10)
fig_class = topic_model.visualize_topics_per_class(topics_per_class)

# Lưu kết quả
topic_model.save(r"../Models/job_topic_model")
fig_barchart.write_html(r"../Figures/topic_keywords.html")
fig_class.write_html(r"../Figures/topics_by_job_role.html")

print(" Hoàn thành, xem file topics_by_job_role.html để thấy sự khác biệt giữa 3 ngành.")

TypeError: check_array() got an unexpected keyword argument 'ensure_all_finite'

In [ ]:
# 1. Lấy thông tin tổng quan về các topic
topic_info = topic_model.get_topic_info()

# 2. Lọc top 15 topic (loại bỏ topic -1 vì đó là các tài liệu bị nhiễu/outliers)
top_15_topics = topic_info[topic_info["Topic"] != -1].head(15)

print(f"{'ID':<5} | {'Số lượng':<8} | {'Từ khóa đặc trưng (Mô tả)':<60}")
print("-" * 80)

for topic_id in top_15_topics["Topic"]:
    # Lấy các từ khóa và trọng số
    words_data = topic_model.get_topic(topic_id)
    
    if not words_data:
        description = "Không có mô tả"
    else:
        # Lấy 7 từ đầu tiên để mô tả rõ hơn về kỹ năng/công cụ
        description = ", ".join([word[0] for word in words_data[:7]])
    
    # Lấy số lượng tài liệu trong topic này
    count = topic_info[topic_info["Topic"] == topic_id]["Count"].values[0]
    
    print(f"{topic_id:<5} | {count:<8} | {description}")

# --- PHẦN BỔ SUNG: Xem topic này thuộc về Chuyên môn nào nhiều nhất ---
print("\n" + "="*30)
print("PHÂN TÍCH TOPIC THEO CHUYÊN MÔN (DA, DS, MLE)")
print("="*30)

# Trích xuất phân bổ topic theo class (đã tạo ở Step 2)
# Dùng để xem top 3 chuyên môn quan tâm đến topic này nhất
for topic_id in top_15_topics["Topic"]:
    # Lấy tên topic (từ khóa đầu tiên)
    topic_label = topic_model.get_topic(topic_id)[0][0]
    
    # Hiển thị phân bổ (dựa trên bảng topics_per_class đã tính ở bước trước)
    relevant_classes = topics_per_class[topics_per_class["Topic"] == topic_id]
    relevant_classes = relevant_classes.sort_values("Frequency", ascending=False).head(3)
    
    dist_str = " | ".join([f"{row['Class']}: {row['Frequency']}" for _, row in relevant_classes.iterrows()])
    print(f"Topic {topic_id} ({topic_label}): {dist_str}")

Topic -1 → không, có, neutral, mà, mua
Topic 0 → pro, 15, 16, 14, 13
Topic 1 → s23, ultra, s24, s25, s23u
Topic 2 → bác, không, đâu, mà, gì
Topic 3 → iphone, android, apple, ip, samsung
Topic 4 → neutral, tiền, không, có, mà
Topic 5 → video, camera, chụp, review, ảnh
Topic 6 → may, co, ban, quá, đầu
Topic 7 → sạc, pin, nhanh, không, dùng
Topic 8 → fold, flip, gập, n3, find
Topic 9 → x8, find, oppo, pro, x7
Topic 10 → oppo, vivo, hơn, chụp, nó
Topic 11 → x200, vivo, mini, pro, x200pro
Topic 12 → xiaomi, phần, không, nó, hãng
Topic 13 → samsung, sam, sony, sung, samfan
